<a href="https://www.kaggle.com/code/jobaerislam/lung-colon-v2?scriptVersionId=238405705" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import libraries
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


# Set your correct dataset directory
dataset_dir = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set'

# Define the image dimensions
img_height, img_width = 224, 224

# Load and preprocess the data
def load_data(dataset_dir):
    images = []
    labels = []

    # List the main folders
    main_folders = ['colon_image_sets', 'lung_image_sets']

    for main_folder in main_folders:
        main_path = os.path.join(dataset_dir, main_folder)
        for folder in os.listdir(main_path):
            label = folder
            folder_path = os.path.join(main_path, folder)
            for filename in os.listdir(folder_path):
                img = cv2.imread(os.path.join(folder_path, filename))
                img = cv2.resize(img, (img_height, img_width))
                images.append(img)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_data(dataset_dir)

# Encode labels (Map class names to numbers)
label_dict = {
    'colon_aca': 0, 
    'colon_n': 1, 
    'lung_aca': 2, 
    'lung_n': 3, 
    'lung_scc': 4
}
labels = np.array([label_dict[label] for label in labels])
labels = to_categorical(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)


# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom top layers for classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(5, activation='softmax')  # 5 classes
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with EarlyStopping
model.fit(X_train, y_train, batch_size=32, epochs=50,
          validation_data=(X_test, y_test),
          callbacks=[early_stop])

2025-05-07 16:17:22.130545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746634642.316057      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746634642.377077      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1746635069.934999      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


I0000 00:00:1746635114.855782      89 service.cc:148] XLA service 0x7d7a080d9890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746635114.856572      89 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1746635118.121726      89 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1746635123.501826      89 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746635123.699094      89 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  1/547 ━━━━━━━━━━━━━━━━━━━━ 8:17:36 55s/step - accuracy: 0.2812 - loss: 1.8275

I0000 00:00:1746635134.276147      89 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


546/547 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9170 - loss: 0.2480

E0000 00:00:1746635181.032385      91 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746635181.231719      91 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


547/547 ━━━━━━━━━━━━━━━━━━━━ 126s 130ms/step - accuracy: 0.9171 - loss: 0.2477 - val_accuracy: 0.3376 - val_loss: 36.0051
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9753 - loss: 0.0741 - val_accuracy: 0.3191 - val_loss: 37.1326
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9781 - loss: 0.0750 - val_accuracy: 0.3844 - val_loss: 16.4683
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9813 - loss: 0.0607 - val_accuracy: 0.1976 - val_loss: 31.1176
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9889 - loss: 0.0349 - val_accuracy: 0.3984 - val_loss: 23.5085
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9892 - loss: 0.0322 - val_accuracy: 0.2572 - val_loss: 21.4297
Epoch 7/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9880 - loss: 0.0336 - val_accuracy: 0.3823 - val_loss: 4.3647
Epoch 8/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.9891 - loss: 0.0317 - val